In [1]:
import pandas as pd
import numpy as np

In [2]:
# Create dataframe from the training datset file
df_train = pd.read_csv('datasets/train3.csv', header='infer', low_memory=False)

# Create dataframe from the testing datset file
df_test = pd.read_csv('datasets/test3.csv', header='infer', low_memory=False)

In [3]:
# Check if dataframes looks good
df_train

df_test

,ID,Text
0,178,!:)I could pass out here and survive the zombi...
1,344,15% tips sneaked into our bill even though we ...
2,2324,How bad could it be? I thought After all its...
3,3217,Jordan is awesome! I have a lot of muscle tens...
4,3705,Micheals Arts and CraftsWell they live under ...
...,...,...
13995,6662447,wifi doesnt work Also even it works it is v...
13996,6662575,wish I could select zero stars This place wa...
13997,6664830,ya know this place is so good and so unassumi...
13998,6665591,you definitely dont want to bring a first date...


In [4]:
# View all columns and their data types
df_train.info()

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    56000 non-null  object
 1   Class   56000 non-null  object
 2   ID      56000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 875.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14000 entries, 0 to 13999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      14000 non-null  int64 
 1   Text    14000 non-null  object
dtypes: int64(1), object(1)
memory usage: 164.1+ KB


In [5]:
# Remove ID columns from both dataframes
del df_train['ID']
del df_test['ID']

In [ ]:
# Write out the pr processed training and test dataframes to a csv file
# df_train.to_csv('processed_training_data.csv', index=False)
# df_test.to_csv('processed_test_data.csv', index=False)

# Done